In [69]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [18]:
cal = pd.read_csv('C:/Users/DEBASHIS/Desktop/E-2-E Calory count prediction using XGboost/notebooks/data/calories.csv')
exc = pd.read_csv('C:/Users/DEBASHIS/Desktop/E-2-E Calory count prediction using XGboost/notebooks/data/exercise.csv')

In [19]:
df = cal.merge(exc, on='User_ID')
df.head()

,User_ID,Calories,Gender,Age,Height,Weight,Duration,Heart_Rate,Body_Temp
0,14733363,231.0,male,68,190.0,94.0,29.0,105.0,40.8
1,14861698,66.0,female,20,166.0,60.0,14.0,94.0,40.3
2,11179863,26.0,male,69,179.0,79.0,5.0,88.0,38.7
3,16180408,71.0,female,34,179.0,71.0,13.0,100.0,40.5
4,17771927,35.0,female,27,154.0,58.0,10.0,81.0,39.8


In [20]:
df.drop(['User_ID'], axis=1, inplace=True)
df.head()


,Calories,Gender,Age,Height,Weight,Duration,Heart_Rate,Body_Temp
0,231.0,male,68,190.0,94.0,29.0,105.0,40.8
1,66.0,female,20,166.0,60.0,14.0,94.0,40.3
2,26.0,male,69,179.0,79.0,5.0,88.0,38.7
3,71.0,female,34,179.0,71.0,13.0,100.0,40.5
4,35.0,female,27,154.0,58.0,10.0,81.0,39.8


In [22]:
encoder = LabelEncoder()
df['Gender'] = encoder.fit_transform(df['Gender'])

In [27]:
df.head()

,Calories,Gender,Age,Height,Weight,Duration,Heart_Rate,Body_Temp
0,231.0,1,68,190.0,94.0,29.0,105.0,40.8
1,66.0,0,20,166.0,60.0,14.0,94.0,40.3
2,26.0,1,69,179.0,79.0,5.0,88.0,38.7
3,71.0,0,34,179.0,71.0,13.0,100.0,40.5
4,35.0,0,27,154.0,58.0,10.0,81.0,39.8


In [34]:
y = df['Calories']
X = df.drop(['Calories'], axis=1)

In [35]:
X.dtypes

Gender          int32
Age             int64
Height        float64
Weight        float64
Duration      float64
Heart_Rate    float64
Body_Temp     float64
dtype: object

In [64]:
columns_to_scale = X.select_dtypes(exclude=['object']).columns
columns_to_scale = columns_to_scale.drop(['Gender'])
columns_to_scale

Index(['Age', 'Height', 'Weight', 'Duration', 'Heart_Rate', 'Body_Temp'], dtype='object')

In [42]:

standard_scaler = StandardScaler()
X[columns_to_scale] = standard_scaler.fit_transform(X[columns_to_scale])

In [44]:
X.head()

,Gender,Age,Height,Weight,Duration,Heart_Rate,Body_Temp
0,1,1.484726,1.089582,1.265909,1.619127,0.989404,0.994023
1,0,-1.342179,-0.593726,-0.995458,-0.183990,-0.158461,0.352342
2,1,1.543620,0.318066,0.268247,-1.265861,-0.784569,-1.701035
3,0,-0.517665,0.318066,-0.263839,-0.304198,0.467647,0.609015
4,0,-0.929922,-1.435380,-1.128480,-0.664821,-1.515029,-0.289338


In [86]:
standard_scaler.var_

array([288.31014929, 203.28025098, 226.05590218,  69.20453031,
        91.83405652,   0.60715879])

In [87]:
columns_to_scale

Index(['Age', 'Height', 'Weight', 'Duration', 'Heart_Rate', 'Body_Temp'], dtype='object')

In [89]:
gender_pipeline = Pipeline(steps=[
    ('encoder', OneHotEncoder())
])

In [90]:
num_pipeline = Pipeline(steps=[
    ('scaler', StandardScaler())
])

In [91]:
pre_processor = ColumnTransformer(transformers=[
    ('age', gender_pipeline, ['Gender']),
    ('num', num_pipeline, columns_to_scale)
])

In [92]:
X,y = df.drop(['Calories'], axis=1), df['Calories']

In [99]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [100]:
X_train = pre_processor.fit_transform(X_train)

In [101]:
X_test

,Gender,Age,Height,Weight,Duration,Heart_Rate,Body_Temp
11499,0,45,154.0,52.0,26.0,107.0,40.6
6475,1,21,187.0,90.0,29.0,111.0,40.5
13167,1,58,176.0,77.0,11.0,90.0,40.0
862,1,35,182.0,89.0,24.0,108.0,40.8
5970,0,67,171.0,67.0,29.0,116.0,41.1
...,...,...,...,...,...,...,...
6216,0,61,166.0,66.0,28.0,106.0,41.0
3585,0,73,165.0,66.0,10.0,92.0,40.0
10023,0,38,169.0,66.0,20.0,104.0,40.2
14044,0,25,163.0,54.0,5.0,86.0,39.2


In [104]:
X_test = pre_processor.transform(X_test)
X_test

array([[ 1.        ,  0.        ,  0.13472117, ...,  1.26666332,
         1.20796311,  0.74304164],
       [ 0.        ,  1.        , -1.28132619, ...,  1.62801363,
         1.62724613,  0.61448958],
       [ 0.        ,  1.        ,  0.90174683, ..., -0.54008823,
        -0.57398972, -0.02827074],
       ...,
       [ 1.        ,  0.        , -0.27829264, ...,  0.5439627 ,
         0.89350085,  0.22883339],
       [ 1.        ,  0.        , -1.0453183 , ..., -1.26278885,
        -0.99327274, -1.05668726],
       [ 0.        ,  1.        , -0.98631632, ...,  1.62801363,
         2.15134991,  1.51435403]])

In [105]:
X_train.shape

(12000, 8)

In [107]:
X_train = pd.DataFrame(X_train,columns=pre_processor.get_feature_names_out())
X_train.head()

,age__Gender_0,age__Gender_1,num__Age,num__Height,num__Weight,num__Duration,num__Heart_Rate,num__Body_Temp
0,0.0,1.0,-0.337295,0.323232,0.139551,-1.021889,-1.517377,-0.671031
1,0.0,1.0,-1.163322,1.447924,0.805955,1.266663,1.522425,0.614490
2,0.0,1.0,-0.573303,0.463818,0.139551,-0.419638,-0.783631,0.100281
3,1.0,0.0,1.373763,-1.293513,-1.393179,-0.780988,-1.936660,-0.671031
4,1.0,0.0,-0.632304,-2.137032,-1.726381,-1.262789,-0.573990,-1.313791


In [118]:
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error


import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square



models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet(),
    'XGBoost':XGBRegressor(objective ='reg:squarederror', learning_rate = 0.1, max_depth = 5, n_estimators = 1000)
}

trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    #this is a validation(test) score
    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')

c:\Users\DEBASHIS\Desktop\E-2-E Calory count prediction using XGboost\env\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
c:\Users\DEBASHIS\Desktop\E-2-E Calory count prediction using XGboost\env\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but Lasso was fitted with feature names
  warnings.warn(
c:\Users\DEBASHIS\Desktop\E-2-E Calory count prediction using XGboost\env\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but Ridge was fitted with feature names
  warnings.warn(
c:\Users\DEBASHIS\Desktop\E-2-E Calory count prediction using XGboost\env\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but ElasticNet was fitted with feature names
  warnings.warn(


LinearRegression
Model Training Performance
RMSE: 11.491913818039597
MAE: 8.4401162109375
R2 score 96.72767822760477


Lasso
Model Training Performance
RMSE: 12.561925581812135
MAE: 9.144387413268921
R2 score 96.08993737141111


Ridge
Model Training Performance
RMSE: 11.489208488752503
MAE: 8.44118417856643
R2 score 96.7292187309627


Elasticnet
Model Training Performance
RMSE: 20.008661605365212
MAE: 15.301911491214653
R2 score 90.0800985014581


XGBoost
Model Training Performance
RMSE: 1.1717763835126425
MAE: 0.8244230702122053
R2 score 99.96597792773314




In [119]:
r2_list

[0.9672767822760477,
 0.960899373714111,
 0.967292187309627,
 0.900800985014581,
 0.9996597792773314]